# Local Weather Impact on Rideshare Metrics

Authors: <br>
Cody French (cmfrench) <br>
Michael Vizzini (mvizzini)

In [ ]:
%pip install -q sodapy

# Data Manipulation Libraries
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
from sodapy import Socrata
import random
random.seed(42)

# Plotting Libraries
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Misc Libraries
from scipy.stats import t
from math import sqrt
import urllib3, socket
from urllib3.connection import HTTPConnection

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 1.8 MB/s eta 0:00:00


In [ ]:
# Remove warnings - example would be when using inplace and modifying the original object as opposed to creating a copy
import warnings

warnings.simplefilter(action="ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Functions

In [ ]:
def getCDAdataset():
    """
        Input: None
        Output: None

        This function makes a connection with the CDA API
        It gathers 1000 samples for each day (same random samples day over day),
        combines, and outputs that data to a csv
    """
    HTTPConnection.default_socket_options = ( 
        HTTPConnection.default_socket_options + [
            (socket.SOL_SOCKET, socket.SO_SNDBUF, 1000000), #1MB in byte
            (socket.SOL_SOCKET, socket.SO_RCVBUF, 1000000)
            ]
        )
    
    start = datetime.strptime("2019-01-01", "%Y-%m-%d")
    end = datetime.strptime("2023-01-01", "%Y-%m-%d")
    date_generated = [start + timedelta(days=x) for x in range(0, (end-start).days)]
    date_lst = [date.strftime("%Y-%m-%d") for date in date_generated]
    
    client = Socrata("data.cityofchicago.org", 
                     "gNV1AerwCbv8s6vkgtnJOMAke",
                     username="cmfrench@umich.edu",
                     password="Bigdatabois23!")
    
    # Results, returned as JSON from API / converted to Python list of
    # dictionaries by sodapy.
    # rand_selection_results = random.choices(results,k=1000)
    # df_trial = pd.DataFrame.from_records(rand_selection_results)
    
    full_result = []
    ride_count = []
    
    for i in range(len(date_lst)):
        if i == (len(date_lst)-1):
            break
        else:
            results = client.get("m6dm-c72p",limit=1000000, where="trip_start_timestamp between '{}' and '{}'".format(date_lst[i],date_lst[i+1]))
            ride_count.append([date_lst[i],len(results)])
            rand_selection = random.choices(results,k=1000)
            full_result += rand_selection
            
    # Convert to pandas DataFrame
    df_rides = pd.DataFrame.from_records(full_result)
    df_rides_count = pd.DataFrame(ride_count, columns =['DATE', 'ride_count'])
    
    with open('/content/drive/My Drive/Milestone I Project/Data/rides.csv', 'w') as f:
        df_rides.to_csv(f)
        
    with open('/content/drive/My Drive/Milestone I Project/Data/rides_count.csv', 'w') as f:
        df_rides_count.to_csv(f)
    
    return None

In [ ]:
def convertToDateTime (df, list_cols):
    """
        Input: Dataframe 
               Column(s) to convert to datetime
        Outpout: None
    """

    for col in list_cols:
        df[col] = pd.to_datetime(df[col])
    #end

    return None

In [ ]:
def makeBin(df, column, newColName, binList, binNameList):

    """
        Input: 
            df - dataframe
            column - column that you would like to bin
            newColName - new column name that is created for groupby
            binList - a list to discretely break the column down
            binNameList - a list of bin names
                          Must have 1 additional element beyond binList

            ex: makeBin(df, "TAVG", "TAVG_TempBin" 
                        [0, 32, 64, 96],
                        ["Very Cold", "Cold", "Nice", "Hot", "Very Hot"]
        
        Output:
            None
    """

    # If the user gives an incorrect number of categorical names
    # based on the number of bins they provide, throw an error
    if len(binNameList) != (len(binList) + 1):
        print("""
        Error: binList is {} elements long, binNameList is {}
        elements long. binNameList needs to be 1 element greater than binList.
        """.format(len(binList), len(binNameList)))

        return None

    # Assign the appropriate bin name
    for i, name in enumerate(binNameList):
        
        if i == 0:
            df.loc[df[column] < binList[i], newColName] = binNameList[i]
        
        elif i < (len(binList) - 1):
            df.loc[(df[column] < binList[i]) & 
                   (df[column] >= binList[i-1]), newColName] = binNameList[i]
        
        else:
            df.loc[df[column] >= binList[i-1], newColName] = binNameList[i]
    
    # Turn the new created variable into a categorical type and set the order
    # for future groupby's

    df[newColName] = df[newColName].astype("category")
    cats = binNameList
    df[newColName] = pd.Categorical(df[newColName], categories = cats)

    return None

## Motivation

Ridesharing has become a consistent part of everyday life for most commuters, but understanding how weather impacts these commutes is something that remains mystery for most. From ride pricing to distances traversed and everything in between, how does weather effect ridesharing.

Both authors of this report are midwesterners, familiar with commuting through all different weather conditions. From hot summer months to bitter cold winter months transportation choices evolve.

Previous studies have attempted to address this question, and certainly helped to shape the type of questions that we proposed for our analysis. In the study "An empirical analysis of taxi, Lyft and Uber rides: Evidence from weather shocks in NYC" author Abel Brodeur writes "Using all taxi, Lyft and Uber rides in New York City, we show that the number of Uber and Lyft rides is significantly correlated with whether it rained. The number of Uber (Lyft) rides per hour is about 22 (19)% higher when it is raining, while the number of taxi rides per hour increases by only 5% in rainy hours-suggesting that surge pricing (prime time) encourages an increase in supply." [1] This study focuses primarily on rain and its effect on ride frequency. We decided that looking more broadly at temperature (and other weather related attributes) and its effect on rideshare pricing would be an interesting next step for this analysis.

That is where the authors of this project wanted to ask some interesting questions to see if different rideshare metrics correlated with weather data. We narrowed our scope to one major city, Chicago Illinois. Utilizing rideshare data provided by the Chicago Data Portal (CDA) and weather data provided by the National Oceanic and Atmosphereic Administration (NOAA) we attempted to answer some questions:

1. Do changes in weather affect ride fare?
    - We have both observed that on colder days, it seems like the cost of various rideshare services increases.
2. Do changes in weather affect average rideshare distance travelled?
    - Are people less willing to walk short distances when the weather is treacherous?
3. Do changes in weather affect ride frequency?
    - During inclement weather, are more people using rideshare services?

These questions provided a starting point for us to deliver an analysis.

## Data Sources

### NOAA Dataset

The National Oceanic and Atmospheric Administration (NOAA) dataset was captured by downloading it from their website: 
https://www.ncei.noaa.gov/cdo-web/datasets/GHCND/locations/CITY:US170006/detail No API was needed and the data was very easy to download using the NOAA's "shopping cart" method.

The data is returned in a *.csv format and was able to be saved in a convenient filepath to be used for this project. The data was only around 8 MB in total size so the full datafile is attached for you to look at if you wish 
(weather.csv).

The NOAA website has extensive documentation regarding what certain columns represent. We were able to check/uncheck certain parameters that we felt would be beneficial for this project directly from the dataset as well as by using "df.columns" after invoking "pd.read_csv()". A list of the important columns as well as their descriptions is provided below.

Columns of Interest:
- STATION: Station Code
- DATE: Date of record
- AWND: Average Wind Speed (MPH)
- PRCP: Precipitation (in to the nearest hundredth)
- SNOW: Snowfall (in to the nearest tenth)
- TAVG: Average Daily Temperature (deg F)
- TMAX: Max Daily Temperature (deg F)
- TMIN: Min Daily Temperature (deg F)

The time period covered in this dataset is from January 1st, 2019 to December 31st, 2022.

### CDA Dataset

The Chicago Data Portal (CDA) dataset was captured by utilizing the Socrata Open Data API (SODA). 

CDA Dataset: https://data.cityofchicago.org/Transportation/Transportation-Network-Providers-Trips/m6dm-c72p
SODA API Documentation: https://dev.socrata.com/foundry/data.cityofchicago.org/m6dm-c72p

The data was recovered using the function shown below which turned JSON formatted infomration (gathered via an API) to a pandas dataframe which was eventually turned to a *.csv format through the use of the "df.to_csv()". These files were saved and stored in a convenient filepath to use for this project. The raw data was around 75 GB which posed challenges in and of itself.

The CDA wesbite has extensive documentation regarding what certain columns represent. Using this documentation along with methods referenced above (df.columns/pd.read_csv()) we were able to narrow the columns of interest down to the following:

Columnns of Interest:
- Trip Start Timestamp: Start of trip timestamp rounded to the nearest 15 minutes
- Trip End Timestamp: End of trip timestamp rounded to the nearest 15 minutes
- Trip Seconds: Duration of trip in seconds
- Trip Miles: Distance covered by trip in miles
- Fare: Base cost of the trip
- Tip: Tip recieved by the driver for the trip
- Additional Charges: Any miscellaneous charges incurred by the customer
- Trip Total: Total charged to the customer
- Trips Pooled: indicates that a customer chose to share a ride with another cusomter
- Pickup Centroid Location: Pickup Location (longitude, latitude) - approximate
- Dropoff Centroid Location: Dropoff Location (longitude, latitude) - approximate

Note: Time information is rounded to the nearest 15 minutes and geographic information is intentionally imprecise to help protect rider privacy. Read more here: http://dev.cityofchicago.org/open%20data/data%20portal/2019/04/12/tnp-taxi-privacy.html

In order to efficently gather data, without bogging down Google Colab (used for this project between the two authors) it was determined that a sample of the rides for each day would be captured (**1000** rides/day). We decided that we would make seperate calls to the API for each day, through a seeded random sample (such that each day has roughly similiar time stamped rides to compare against).

The time period covered in this dataset is from January 1st, 2019 to December 31st, 2022.

In [ ]:
# This function was used to capture all of the data from the CDA dataset
# WARNING: This takes a very long time to execute!
# A shorted dataset is attached with a sample of the entire dataset!

# getCDAdataset()

## Data Manipulation Methods

### NOAA Dataset

When read in, the NOAA dataset was near ready for immediate analysis, however some additional steps were taken and assumptions were made regarding the dataset.

Assumptions: 
- Weather is a slow moving, locally ubiquitous phenomenom.
- Chicago is a city without too much sprawl.
- Weather data captured by a single, reliable station can be applied to the entire city of Chicago on any given day.

With these assumptions in place, we turned our attention to finding a highly reliable station (both in timing of data captured, as well as signals of interest).

In [ ]:
"""
Columns of Interest:
    STATION: Station Code
    DATE: Date of record
    AWND: Average Wind Speed (MPH)
    PRCP: Precipitation (in)
    SNOW: Snowfall (in)
    TAVG: Average Daily Temperature (deg F)
    TMAX: Max Daily Temperature (deg F)
    TMIN: Min Daily Temperature (deg F)
"""

df_weather = pd.read_csv('/content/drive/My Drive/Milestone I Project/weather.csv')

So let's try to find our most reliable weather station that meets our assumptions.

In [ ]:
# cols_weather is list of all of the columns of interest outlined above
# let's reduce our dataframe down to just those columns
cols_weather = ['STATION', 'DATE', 'TAVG','TMAX','TMIN', 'AWND','PRCP','SNOW']
df_weather_cut = df_weather[cols_weather]

# Next lets perform a split-apply-combine
# SPLIT - on a given station
# APPLY - count the number of non-nan values
# COMBINE - our results back into a single dataframe
df_weather_station = df_weather_cut.groupby('STATION').count()

# Sort the dataframe - the order of the list we made above is the same order that the sort will do!
# Since 'STATION' is what is grouped, 'DATE' will be sorted first where all ties will then
# be broken by 'TAVG', followed by 'TMAX', and so on
df_weather_station.sort_values(by=cols_weather, ascending=False, inplace=True)

# Let's print out the head of this dataframe!
df_weather_station.head()

,DATE,TAVG,TMAX,TMIN,AWND,PRCP,SNOW
STATION,,,,,,,
USW00094846,1461,1461,1461,1461,1461,1461,1461
USW00014819,1461,0,1461,1461,1460,1457,207
USW00004838,1461,0,1460,1458,1460,1455,209
USC00116616,1459,0,1445,1447,0,1436,1279
USC00115110,1452,0,776,826,0,1422,1445


Station "USW00094846" seems very reliable for our purposes! Not only is every single date recorded (even the 4th most consistent station missed 2 days during this period!) it also consistently captured weather metrics of interest to us! I wonder what the name of this weater station is?

In [ ]:
df_weather["NAME"][df_weather['STATION'] == "USW00094846"].iloc[0]

'CHICAGO OHARE INTERNATIONAL AIRPORT, IL US'

Chicago O'Hare International Airport is our reliable weather station! This makes perfect sense! We would expect that an international airport must capture crucial data in order to maintain safety.

So let's take our original dataframe and cut it down to only the columns of interest where the station is associated with Chicago O'Hare International Airport.

In [ ]:
# Use our list of important columns as stated above
# Use boolean masking to reduce our dataset to only data recorded by
# Chicago O'Hare International Airport
dfWeather = df_weather[cols_weather]
dfWeather = dfWeather[dfWeather["STATION"] == "USW00094846"]

Now let's take a high level view of our resulting dataframe to see some information about the dataframe itself using the df.info() function that is built into pandas.

In [ ]:
dfWeather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1461 entries, 83436 to 84896
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   STATION  1461 non-null   object 
 1   DATE     1461 non-null   object 
 2   TAVG     1461 non-null   float64
 3   TMAX     1461 non-null   float64
 4   TMIN     1461 non-null   float64
 5   AWND     1461 non-null   float64
 6   PRCP     1461 non-null   float64
 7   SNOW     1461 non-null   float64
dtypes: float64(6), object(2)
memory usage: 102.7+ KB


We see that our columns of interest are shown, and we can also see that every column contains some non-null data. Finally we can see the datatype of the various columns.

The only column that needs to be changed is the *DATE* column. Let's change this to a datetime using a function we have created above in the "Functions" section. We will also call df.info() again to verify that things changed as expected.

In [ ]:
# convertToDateTime takes in a list of columns that you want to convert to pd.datetime
convertToDateTime(dfWeather, ['DATE'])

dfWeather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1461 entries, 83436 to 84896
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   STATION  1461 non-null   object        
 1   DATE     1461 non-null   datetime64[ns]
 2   TAVG     1461 non-null   float64       
 3   TMAX     1461 non-null   float64       
 4   TMIN     1461 non-null   float64       
 5   AWND     1461 non-null   float64       
 6   PRCP     1461 non-null   float64       
 7   SNOW     1461 non-null   float64       
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 102.7+ KB


As a gut check we can do a couple of things:
1. Print out a df.describe() of the dataframe to make sure that there are no extreme outliers
2. Plot one of our metrics so that we can get a base understanding about how some of the more cruical elements of our dataframe look

In [ ]:
# Method 1: Utilize df.describe() to look at some metrics regarding the dataframe
dfWeather.describe()

,TAVG,TMAX,TMIN,AWND,PRCP,SNOW
count,1461.000000,1461.000000,1461.000000,1461.000000,1461.000000,1461.000000
mean,51.958932,60.330595,43.683094,9.605168,0.103285,0.106845
std,19.922891,21.234718,19.255532,3.489934,0.288226,0.545562
min,-15.000000,-10.000000,-23.000000,2.240000,0.000000,0.000000
25%,36.000000,43.000000,30.000000,7.160000,0.000000,0.000000
50%,52.000000,61.000000,42.000000,9.170000,0.000000,0.000000
75%,70.000000,79.000000,62.000000,11.630000,0.050000,0.000000
max,89.000000,99.000000,81.000000,27.960000,3.530000,6.300000


All of these metrics look reasonable which makes our job that much easier. It looks like the temperature assocaiated with Chicago is temperate (betwen 43 and 60 degrees F). It's not called the "Windy City" for nothing, with an average wind speed of around 9.5 MPH. The "max" and "min" rows appear to hold reasonable numbers as well. Let's move on to graphing a metric like *TAVG* to see how it has changed over the years.

In [ ]:
# Method 2: Visualize a metric over time to see if any visual unusual cues jump out at us
fig = px.histogram(
    dfWeather,
    x="DATE",
    y="TAVG",
    nbins=len(dfWeather),
    title="Average Temperature Over Full Time Window"
)

fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Temperature (Degrees F)",
    plot_bgcolor='rgba(0,0,0,0)',
    height=600,
    width=800
)

fig.show()

Average Temperature appears to be following a nice seasonal pattern that looks to be expected. By visualizing the data beyond just printing out a df.describe() we are able to more effectively understand what data we are working with and see if any interesting trends like this one exist!

Finally, let's print out the head of the resulting dataframe!

In [ ]:
dfWeather.head()

,STATION,DATE,TAVG,TMAX,TMIN,AWND,PRCP,SNOW
83436,USW00094846,2019-01-01,31.0,34.0,26.0,7.61,0.01,0.1
83437,USW00094846,2019-01-02,28.0,30.0,26.0,8.72,0.01,0.1
83438,USW00094846,2019-01-03,29.0,38.0,24.0,11.63,0.00,0.0
83439,USW00094846,2019-01-04,37.0,48.0,31.0,8.05,0.00,0.0
83440,USW00094846,2019-01-05,39.0,52.0,32.0,8.28,0.00,0.0


### CDA Dataset

After assembling our dataset from the API, it required only minimal cleaning. As with the NOAA dataset, we made a few assumption that would guide the cleaning of the dataset.
Assumptions:
- Exact location was not pertinant to our analysis, we are interested in Chicago as a whole
- The start date of the ride would be considered as the primary date of the ride. (ie. a ride starting at 11:50 pm and ending at 12:05 am would be classified as the ride on the day it started)

In [ ]:
"""
Columnns of Interest:
    Trip Start Timestamp: Start of trip timestamp
    Trip End Timestamp: End of trip timestamp
    Trip Seconds: Duration of trip in seconds
    Trip Miles: Distance covered by trip in miles
    Fare: Base cost of the trip
    Tip: Tip recieved by the driver for the trip
    Additional Charges: Any miscellaneous charges incurred by the customer
    Trip Total: Total charged to the customer
    Trips Pooled: indicates that a customer chose to share a ride with another cusomter
"""

df_rides = pd.read_csv('/content/drive/My Drive/Milestone I Project/Data/rides.csv')

In [ ]:
# cols_rides is a list of all the columsn of interest outlined above
# let's reduce our datafrom down to just those columns
cols_rides = ['trip_start_timestamp', 'trip_end_timestamp', 'trip_seconds',
              'trip_miles', 'fare', 'tip', 'additional_charges', 'trip_total',
              'trips_pooled']

dfRides = df_rides[cols_rides]

Now let's take a high level view of our resulting dataframe to see some information about the dataframe isself and each of our varaibles of interest using the df.info() function built into pandas.

In [ ]:
dfRides.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461000 entries, 0 to 1460999
Data columns (total 9 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   trip_start_timestamp  1461000 non-null  object 
 1   trip_end_timestamp    1461000 non-null  object 
 2   trip_seconds          1460282 non-null  float64
 3   trip_miles            1460972 non-null  float64
 4   fare                  1460150 non-null  float64
 5   tip                   1460150 non-null  float64
 6   additional_charges    1460150 non-null  float64
 7   trip_total            1460150 non-null  float64
 8   trips_pooled          1461000 non-null  int64  
dtypes: float64(6), int64(1), object(2)
memory usage: 100.3+ MB


We know that our dataset encompasses 4 years (2019-2022), which should be 1461 days. We took a sample of 1000 rides per day (each day is actually around 1000 rides per day, +/- 19 rides. This could be a result of the random function we invoked when calling to the API) so we can expect 1,461,000 data entries. This is communicated in the range index of df.info(). So the shape of our resulting dataframe looks appropriate.

Some columns contain some rows that are null. *fare*, *tip*, *additional_charges*, and *trip_total* contain the most null information.

Before we simply drop all rows that contain null information, we should check to make sure that a single day is not the culprit for these missing values (if we drop all rows without checking, it is possible that all of the null pieces of financial information came from a single day!)

In [ ]:
# Let's start converting our time columns of interest to the appropriate datatype
cols2date = ['trip_start_timestamp', 'trip_end_timestamp']
convertToDateTime(dfRides, cols2date)

# Next, let's collapse our time indicator used for grouping "trip_start_timestamp"
# down to only capture the year, month, and day information
dfRides['DATE'] = pd.to_datetime(dfRides['trip_start_timestamp'].dt.strftime('%Y-%m-%d'))

# Finally, let's perform a Split-Apply-Combine on the newly created "DATE" column
# From here can perform an operation similiar to what was done above and filter
# our search by days with the least amount of information
dfRides_Grouped = dfRides.groupby('DATE').count()
dfRides_Grouped.sort_values(by=cols_rides, inplace=True)
dfRides_Grouped.head()

,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,fare,tip,additional_charges,trip_total,trips_pooled
DATE,,,,,,,,,
2022-09-23,981,981,981,981,978,978,978,978,981
2019-11-08,982,982,982,982,982,982,982,982,982
2021-08-07,983,983,983,983,983,983,983,983,983
2021-09-24,984,984,984,984,983,983,983,983,984
2022-04-22,984,984,984,984,984,984,984,984,984


So we can see that, given the filter order provided by "cols_rides" above, September 23rd 2022 was one of our more sparse days of capturing data, but that only 3 rides out of 981 are missing *fare*, *tip*, *additional_charges*, and *trip_total*. This should not influence our analysis so we can go ahead with the dropping procedure.

We are going to simply use df.dropna() to remove any row containing missing information.

In [ ]:
dfRides.dropna(axis=0, inplace=True)

rides_dropped = 1461000 - dfRides.shape[0]
per = (rides_dropped/1461000)*100

print("Total Rides Dropped: {}".format(rides_dropped))
print("Percentage of Dataframe Dropped: {:.2f}%".format(per))

Total Rides Dropped: 1596
Percentage of Dataframe Dropped: 0.11%


We can see the number of total rides dropped as well as the percentage of the dataframe that we lost by doing this procedure!

Let's now do a df.describe() to see if any outliers jump off the page. If any do we can then go into greater detail regarding how to deal with these strange values.

In [ ]:
dfRides.describe()

,trip_seconds,trip_miles,fare,tip,additional_charges,trip_total,trips_pooled
count,1.459404e+06,1.459404e+06,1.459404e+06,1.459404e+06,1.459404e+06,1.459404e+06,1.459404e+06
mean,1.051594e+03,6.702829e+00,1.520244e+01,8.304095e-01,3.813325e+00,1.984617e+01,1.067865e+00
std,7.453520e+02,7.474271e+00,1.247278e+01,2.305337e+00,3.250140e+00,1.493881e+01,3.784576e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
25%,5.350000e+02,2.000000e+00,7.500000e+00,0.000000e+00,2.550000e+00,1.058000e+01,1.000000e+00
50%,8.570000e+02,4.070000e+00,1.250000e+01,0.000000e+00,3.080000e+00,1.541000e+01,1.000000e+00
75%,1.359000e+03,8.612800e+00,2.000000e+01,0.000000e+00,4.660000e+00,2.374000e+01,1.000000e+00
max,5.700000e+04,3.320300e+02,8.150000e+02,5.000000e+02,2.321500e+02,8.168800e+02,2.000000e+01


Huh? Minimum *trip_miles* is 0.0000 miles and minimum *trip_seconds* is 3 seconds. This seems like noise that we would want to filter out. An example of what this might represent is a driver hitting "Start" and then "End" without actually moving the vehicle. Let's take an educated guess at some kind of filtering that could be applied to remove this noise.

It is also interesting to look at maximum *trip_seconds* and maximum *trip_miles*. The maximum trip seconds equates to just over 5 hours in vehicle while the maximum trip miles is just shy of 190 miles. Chicago is approximately 25 miles long and 15 miles wide. On days where traffic is as bad as it comes, commutes from one corner to the other, opposite corner could take around 2-3 hours potentially from our best guesses. These data points are more difficult to classify as noise, but with these assumptions we can try.

Assumptions: 
- Trips where *trip_miles* < 0.05 are noise (customer has not gone anywhere)
- Trips where *trip_miles* > 100 are noise (rideshare is leaving Chicago)
- Trips where *trip_seconds* < 45 are noise (erronous trip initiation)
- Trips where *trip_seconds* > 10800 are noise (trip > 3 hours; leaving Chicago)

In [ ]:
# Filter the set to only retain rides that were longer than 0.05 miles long
dfRides = dfRides[dfRides["trip_miles"] >= 0.05]

# Filter the set to only retain rides that were 100 miles or less
dfRides = dfRides[dfRides["trip_miles"] <= 100]

# Filter the set to only retain rides that were at least 45 seconds long
dfRides = dfRides[dfRides["trip_seconds"] >= 45]

# Filter the set to only retain rides that were 3 hours or less in duration
dfRides = dfRides[dfRides["trip_seconds"] <= 10800]

In [ ]:
new_rides_dropped = (1461000 - rides_dropped) - dfRides.shape[0]
rides_dropped = 1461000 - dfRides.shape[0]
per = (rides_dropped/1461000)*100

print("New Rides Dropped: {}".format(new_rides_dropped))
print("Total Rides Dropped: {}".format(rides_dropped))
print("Percentage of Dataframe Dropped: {:.2f}%".format(per))

New Rides Dropped: 1047
Total Rides Dropped: 2643
Percentage of Dataframe Dropped: 0.18%


Let's take a final look at the head of our dataframe!

In [ ]:
dfRides.head()

,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,fare,tip,additional_charges,trip_total,trips_pooled,DATE
0,2019-01-01 13:45:00,2019-01-01 13:45:00,485.0,2.619944,5.0,0.0,0.00,5.00,2,2019-01-01
1,2019-01-01 00:15:00,2019-01-01 00:15:00,206.0,0.750686,5.0,0.0,2.50,7.50,1,2019-01-01
2,2019-01-01 02:30:00,2019-01-01 02:45:00,912.0,3.817651,10.0,0.0,2.50,12.50,1,2019-01-01
3,2019-01-01 02:00:00,2019-01-01 02:15:00,757.0,3.413438,7.5,0.0,2.50,10.00,1,2019-01-01
4,2019-01-01 16:15:00,2019-01-01 16:30:00,742.0,4.685210,15.0,0.0,2.85,17.85,1,2019-01-01


Due to the shear size of the dataset, we had to develop a way to create a representative set, which would have left us without a key variable in our analysis, the number of daily rides. When calling the API and assembling the dataset we also created a second set with daily ride counts.

Lets now read in that file and take a look at its format.


In [ ]:
# Read in our ride_counts file created during the API call
# usecols=range(2,4) used to skip over the first two columns: "Unnamed: 0" & "Unnamed: 0.1"
df_rides_counts = pd.read_csv('/content/drive/My Drive/Milestone I Project/Data/rides_count.csv',
                              usecols=range(2,4))
df_rides_counts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461 entries, 0 to 1460
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   DATE        1461 non-null   object
 1   ride_count  1461 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 23.0+ KB


The dataframe has 1461 entries, the number of days in our dataset, as expected but some cleaning needs to be done. We will use our datatime function to convert the *DATE* column in to the correct format. Finally, let's again take a look at the dataframe using df.info().

In [ ]:
convertToDateTime(df_rides_counts, ['DATE'])

df_rides_counts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461 entries, 0 to 1460
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   DATE        1461 non-null   datetime64[ns]
 1   ride_count  1461 non-null   int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 23.0 KB


In [ ]:
df_rides_counts.head()

,DATE,ride_count
0,2019-01-01,298298
1,2019-01-02,214963
2,2019-01-03,228100
3,2019-01-04,280936
4,2019-01-05,300642


It's at this point that the author's made the decision to leave the df_rides_counts and previously worked on dfRides dataframes seperate. We could have duplicated the *ride_counts* integer many times over a newly created column in the dfRides dataframe, but in the next step we are going to merge all of these dataframes together, so without further ado, let's continue on!

### Combining the Datasets

Before combining the dataset we had to determine what variable/column we would like to merge on for our analysis needs. The NOAA dataset has daily weather information and the CDA dataset has rides down to the 15 minute. To analyze pattern in the data, we want to get the ride statistics in a daily format so that the datasets can be merge on date. Thankfully, this step was completed above which allowed us to groupby("DATE") to see if any particular day would be affected by a call to dropna().

In [ ]:
# Preparing the dataset for merge by sorting based on date
dfRides.sort_values(by='DATE',inplace=True)
dfWeather.sort_values(by='DATE',inplace=True)
dfWeather.set_index('DATE',inplace=True)

In [ ]:
# Group rides based on our new date column and apply an aggregation function to get the mean and standard deviation
dfRidesGrouped = dfRides.groupby('DATE').agg(['mean','std'])

# Unstack the column titles
dfRidesGrouped.columns = ['_'.join(cols).lower()
                          for cols in dfRidesGrouped.columns.to_flat_index()]

Now the the Rides dataframe is ready to be merged, we will merge the Rides and ride counts data frames first. Then we will merge this with the Weather dataset.

In [ ]:
# Merge Rides and Rides Counts Datasets
dfmerge = dfRidesGrouped.merge(df_rides_counts, left_on='DATE', right_on='DATE')

# Merge the Rides and Rides Counts Dataset with the Weather Dataset
dfmerge = dfmerge.merge(dfWeather, left_on='DATE', right_on='DATE')

In order to make visualizations and analysis move along nicely, we invoked a function that is stored in the "Functions" section. By giving the function a column of interest we can make categorical outputs given on quantifiable data.

In [ ]:
# Bin creation for Average Temperature
makeBin(dfmerge, "TAVG", "TAVG_Bin", [0, 15, 45, 60], 
        ["Very Cold", "Cold", "Moderate", "Hot", "Very Hot"])

# Bin creation for Snowfall
makeBin(dfmerge, "SNOW", "SNOW_Bin", [0.1, 1, 3], 
         ["No Snow", "Light Snow", "Moderate Snow", "Heavy Snow"])

# Bin creation for Rainfall
makeBin(dfmerge, "PRCP", "PRCP_Bin", [0.01, 0.24, 0.48], 
         ["No Rain", "Light Rain", "Moderate Rain", "Heavy Rain"])

# Bin creation for Average Wind Speed
makeBin(dfmerge, "AWND", "AWND_Bin", [7, 18, 25], 
         ["Light Wind", "Moderate Wind", "Strong Wind", "Heavy Wind"])

In [ ]:
dfmerge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1461 entries, 0 to 1460
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   DATE                     1461 non-null   datetime64[ns]
 1   trip_seconds_mean        1461 non-null   float64       
 2   trip_seconds_std         1461 non-null   float64       
 3   trip_miles_mean          1461 non-null   float64       
 4   trip_miles_std           1461 non-null   float64       
 5   fare_mean                1461 non-null   float64       
 6   fare_std                 1461 non-null   float64       
 7   tip_mean                 1461 non-null   float64       
 8   tip_std                  1461 non-null   float64       
 9   additional_charges_mean  1461 non-null   float64       
 10  additional_charges_std   1461 non-null   float64       
 11  trip_total_mean          1461 non-null   float64       
 12  trip_total_std           1461 non-

In [ ]:
dfmerge.head()

,DATE,trip_seconds_mean,trip_seconds_std,trip_miles_mean,trip_miles_std,fare_mean,fare_std,tip_mean,tip_std,additional_charges_mean,...,TAVG,TMAX,TMIN,AWND,PRCP,SNOW,TAVG_Bin,SNOW_Bin,PRCP_Bin,AWND_Bin
0,2019-01-01,879.679112,570.812143,5.737055,6.087336,10.623108,8.339936,0.464178,1.372338,2.673380,...,31.0,34.0,26.0,7.61,0.01,0.1,Moderate,Light Snow,Light Rain,Moderate Wind
1,2019-01-02,1012.960040,649.485855,6.337168,6.647021,10.714286,8.647357,0.570430,1.545084,2.980669,...,28.0,30.0,26.0,8.72,0.01,0.1,Moderate,Light Snow,Light Rain,Moderate Wind
2,2019-01-03,1024.517034,654.477475,6.067278,5.859285,10.092685,7.308071,0.485972,1.364878,2.774319,...,29.0,38.0,24.0,11.63,0.00,0.0,Moderate,No Snow,No Rain,Moderate Wind
3,2019-01-04,972.835341,622.290988,5.306055,5.429431,9.869478,7.402331,0.483936,1.313533,2.628283,...,37.0,48.0,31.0,8.05,0.00,0.0,Moderate,No Snow,No Rain,Moderate Wind
4,2019-01-05,921.002014,588.990770,5.192315,5.464591,9.896777,7.989451,0.495468,1.399967,2.659325,...,39.0,52.0,32.0,8.28,0.00,0.0,Moderate,No Snow,No Rain,Moderate Wind


## Analysis


After recognizing our data sources and doing some prelimary effort to clean and manipulate the data, we are ready to answer the questions outlined in the "Motiviation" section of the report. Each section below is broken out by the question at hand.

Note, all of these questions utilize the split-apply-combine strategy described by Hadley Wickham. In general the approach used to answer these questions relies on the aggregation of data on a condition. Split means that we will look to find all instances of our merged dataset where some condition is common. Apply refers to performing some kind of aggregation on these grouped, conditionally common sets. And finally, combine refers to the action of taking these refined, typically smaller dataframes, and reinserting them back into a single dataframe which allows for easier comparison and digestion of the information.

### Do Changes in Weather Affect Ride Fare?

Both authors initially assumed that as weather moved away from nice, nominal conditions (ex: moving towards extreme cold or extreme hot, increased snowfall, etc) that rideshare fare would increase.

Let's use our newly created bins to determine what the "main factors" are that influence ride share metrics.

*Note: PCA would be a viable option in determining how much explained variance we can see in our target variable, fare_mean, through our weather metrics (attributes). As PCA is outside of the scope of this project, a simpler approach where range in the target variable is observed to determine which attributes are our lead contributors.*

With four bins created there are 320 different combinations of weather metrics to visualize. To help reduce this we are going to only take the top 2 weather metrics.

In [ ]:
# Go through each of the weather metrics and look at the resulting dataframes range
# Only take the top two weather metrics with the greatest range in fare_mean accounted 
# for within that weather metric

rangeDict = {}

for bin in ["TAVG_Bin", "SNOW_Bin", "PRCP_Bin", "AWND_Bin"]:

    temp_df = dfmerge.groupby(bin).mean()[["fare_mean"]]
    temp_df.sort_values("fare_mean", ascending=False, inplace=True)

    rangeDict[bin] = (temp_df.iloc[0,0] - temp_df.iloc[-1,0])
#end

print("For each weather metric: Maximum 'fare_mean' - Minimum 'fare_mean' ")
rangeDict

For each weather metric: Maximum 'fare_mean' - Minimum 'fare_mean' 


{'TAVG_Bin': 3.1101859160705025,
 'SNOW_Bin': 2.246046114972124,
 'PRCP_Bin': 0.3367225573583301,
 'AWND_Bin': 1.4446421470126456}

We see that *TAVG_Bin* and *SNOW_Bin* account for the greatest range in *fare_mean*, so let's reduce our analysis down to just those weather metrics.

In [ ]:
df_q1 = dfmerge.groupby(["TAVG_Bin", "SNOW_Bin"]).mean()[["fare_mean", "tip_mean"]].reset_index()
smallFare = df_q1.sort_values("fare_mean", ascending=True)[0:5]
bigFare = df_q1.sort_values("fare_mean", ascending=False)[0:5]

Let's show the dataframes "smallFare" and "bigFare" which represent *fare_mean* sorted by smaller values or bigger values which translates to people paying rideshare services a small amount or a big amount, respectively.

In [ ]:
smallFare

,TAVG_Bin,SNOW_Bin,fare_mean,tip_mean
6,Cold,Moderate Snow,9.530470,0.499500
1,Very Cold,Light Snow,10.411647,0.594378
4,Cold,No Snow,13.637769,0.747197
9,Moderate,Light Snow,13.653826,0.758017
8,Moderate,No Snow,13.804411,0.733432


In [ ]:
bigFare

,TAVG_Bin,SNOW_Bin,fare_mean,tip_mean
7,Cold,Heavy Snow,22.315253,0.811035
16,Very Hot,No Snow,16.326427,0.925037
12,Hot,No Snow,15.336822,0.818938
11,Moderate,Heavy Snow,14.780951,0.749330
5,Cold,Light Snow,14.687788,0.622500


In [ ]:
print("Average fare for the entire dataframe is: ${:.2f}".format(dfmerge["fare_mean"].mean()))

Average fare for the entire dataframe is: $15.17


Looking at bigFare, it looks like the highest *fare_mean* occurs when heavy snow is introduced in Chicago, which makes complete sense as snow is difficult to traverse in and poses a safety risk. If *TAVG* is Cold or Moderate then snow is usually involved which again makes sense. Only when *TAVG* is Very Hot or Hot does *fare_mean* move into those high values.

Let's contrast this to smallFare. The cheapest trips are trips where we expect the weather to be nominal and we can see that in *TAVG* being Moderate with light or no snow making the list. Cold conditions look to impact ride share pricing but when there is little snow accumulation, the *fare_mean* price is quite affordable.

As a fun additional observation, take a look at tip_mean. Tipping on Lyft and Uber is completely optional [2], so these tips are unrelated to the *fare_mean* attribute. Customers look to be more generous in worse conditions!

### Do Changes in Weather Affect Average Rideshare Distance Travelled?

"It's freezing out, let's just take an Uber" is a common phrase you might overhear after walking out of a bar or restaurant on a cold winter's night. Would customers have been more willing to walk to their destination if the weather was more favorable? Are people less willing to walk short distances or take public transit when the weather is treacherous? What weather conditions are present for the short trips?

In [ ]:
# Let's perform a series of groupby's

df_q2 = None

for bin in ["TAVG_Bin", "SNOW_Bin", "PRCP_Bin", "AWND_Bin"]:

    temp_df = dfmerge.groupby(bin).mean()[["trip_miles_mean", "trip_seconds_mean"]]
    temp_df["Weather Type"] = bin[:-4]

    if df_q2 is None:
        df_q2 = temp_df
    else:
        df_q2 = pd.concat([df_q2, temp_df])
    #end
#end

df_q2.sort_values('trip_miles_mean',inplace=True)
df_q2

,trip_miles_mean,trip_seconds_mean,Weather Type
Very Cold,5.947973,933.896081,TAVG
Heavy Wind,5.958464,979.971826,AWND
Heavy Snow,6.128677,1025.315277,SNOW
Cold,6.223578,1002.094079,TAVG
Moderate Snow,6.313130,1014.865604,SNOW
Light Snow,6.328169,1019.961600,SNOW
Strong Wind,6.452352,1007.829966,AWND
Heavy Rain,6.482392,1061.695192,PRCP
Moderate,6.499658,1014.672492,TAVG
Moderate Rain,6.570015,1064.126145,PRCP


It looks like our initial assumption was correct! The average rideshare distance decreases during inclement weather. Rideshare distance is shortest when it is cold, snowing, and of course windy! This suggests that people are more willing to rideshare for a trip that they might otherwise walk or use public transportation for when the weather is snowy, cold, rainy, or windy.

### Do Changes in Weather Affect Ride Frequency?
During incliment weather, are more people using rideshare services?

In [ ]:
df_q3 = dfmerge.groupby(['SNOW_Bin','PRCP_Bin','TAVG_Bin','AWND_Bin'])['ride_count'].mean().reset_index().dropna()
most_freq_cond = df_q3.sort_values('ride_count',ascending=False)[0:10]
most_freq_cond

,SNOW_Bin,PRCP_Bin,TAVG_Bin,AWND_Bin,ride_count
184,Moderate Snow,Light Rain,Cold,Light Wind,389629.0
88,Light Snow,No Rain,Moderate,Light Wind,388836.0
104,Light Snow,Light Rain,Cold,Light Wind,373897.0
211,Moderate Snow,Moderate Rain,Moderate,Heavy Wind,373502.0
15,No Snow,No Rain,Hot,Heavy Wind,354939.0
54,No Snow,Moderate Rain,Hot,Strong Wind,336454.0
310,Heavy Snow,Heavy Rain,Moderate,Strong Wind,315280.0
30,No Snow,Light Rain,Moderate,Strong Wind,307345.0
50,No Snow,Moderate Rain,Moderate,Strong Wind,300314.0
100,Light Snow,Light Rain,Very Cold,Light Wind,294455.0


Very interesting, the conditions where rideshares are the most frequent are NOT during the most extreme or incliment conditions. This doesn't quite make sense with what we have seen so far. As mentioned above, there are 320 different combinations of weather metrics. Lets filter our data to remove any combinations of weather that are "rare" (occuring 5 or less times in the set).

In [ ]:
df_q3 = dfmerge.groupby(['SNOW_Bin','PRCP_Bin','TAVG_Bin','AWND_Bin'])['ride_count'].agg(['mean','count']).reset_index().dropna()
df_q3 = df_q3[df_q3['count']>5]
most_freq_cond_c = df_q3.sort_values('mean',ascending=False)[0:10]
most_freq_cond_c

,SNOW_Bin,PRCP_Bin,TAVG_Bin,AWND_Bin,mean,count
69,No Snow,Heavy Rain,Moderate,Moderate Wind,274339.166667,12
89,Light Snow,No Rain,Moderate,Moderate Wind,245580.636364,11
189,Moderate Snow,Light Rain,Moderate,Moderate Wind,231459.470588,17
56,No Snow,Moderate Rain,Very Hot,Light Wind,231257.100000,10
109,Light Snow,Light Rain,Moderate,Moderate Wind,230072.482759,29
53,No Snow,Moderate Rain,Hot,Moderate Wind,223945.086957,23
29,No Snow,Light Rain,Moderate,Moderate Wind,219616.083333,36
5,No Snow,No Rain,Cold,Moderate Wind,217904.526316,19
12,No Snow,No Rain,Hot,Light Wind,207294.073171,41
49,No Snow,Moderate Rain,Moderate,Moderate Wind,207019.428571,14


After filtering we still see that the most rides are on days that have mostly light to moderate weather phenomenon. One explanation of this might be that people prefer to "go out" when the weather is nice and prefer to stay home when it is bad.

Lets also take a look at the correlation between ride counts and our weather variables. 

In [ ]:
df_corr = dfmerge.corr(method="pearson")
df_corr = df_corr[['TAVG', 'TMAX', 'TMIN', 'AWND', 'PRCP', 'SNOW','ride_count']]
df_corr.drop(cols_weather[2:],axis=0,inplace=True)

ride_count_corr = df_corr.loc['ride_count'][:-1]
ride_count_corr

TAVG   -0.143292
TMAX   -0.145693
TMIN   -0.142188
AWND    0.036584
PRCP    0.038382
SNOW    0.004566
Name: ride_count, dtype: float64

There does not seem to be any strong correlations between ride count and our weather variables (all correlations are close to 0). The strongest correlation are negative correlations between temperature and ride count.

Great! We were able to analyze our questions by creating various dataframes and gathering insights from said dataframes, but let's take things a step further and visualize some of these relationships!

## Visualizations

### Heatmap of How Weather Effects Ride Fare

Returning to our first question of "Do changes in weather affect ride fare?", let's create a heatmap of our two categorical bins and plot average ride fare. Remember that *TAVG_Bin* and *SNOW_Bin* were the two weather metrics that explained the greatest variance in the *fare_mean* target variable, so these will be what we choose to plot.

In [ ]:
# Converting the output of df_q1 to a wide format so that we can
# make a heatmap

hm = pd.pivot_table(df_q1,
                    index="SNOW_Bin",
                    columns="TAVG_Bin",
                    values="fare_mean")

hm

TAVG_Bin,Very Cold,Cold,Moderate,Hot,Very Hot
SNOW_Bin,,,,,
No Snow,14.618539,13.637769,13.804411,15.336822,16.326427
Light Snow,10.411647,14.687788,13.653826,NaN,NaN
Moderate Snow,NaN,9.530470,14.273043,NaN,NaN
Heavy Snow,NaN,22.315253,14.780951,NaN,NaN


In [ ]:
fig = px.imshow(hm,
                color_continuous_scale="Aggrnyl",
                text_auto=".2f")

fig.update_layout(title="Average Fare by Weather Conditions",
                  xaxis={"title": "Average Temperature Binned"},
                  yaxis={"title": "Snowfall Accumulation Binned"},
                  coloraxis_colorbar={"title": "Mean Fare ($USD)"},
                  plot_bgcolor="rgba(0,0,0,0)",
                  width=1000,
                  height=800)
fig.show()

### Bar Chart of How Weather Effects Rideshare Distance Travelled

Returning to our second question of "Do changes in weather affect average rideshare distance travelled?", let's create a bar chart that breaks out each of the metrics and plots "trip_miles_mean".

In [ ]:
fig = px.bar(df_q2,
             x="trip_miles_mean",
             y=df_q2.index,
             labels={"index":"Weather Metric",
                     "trip_miles_mean": "Average Distance Travelled (Miles)"},
             text_auto=".2f",
             color="Weather Type")

fig.update_traces(text=df_q2["trip_miles_mean"],
                  textposition="inside")

fig.update_layout(yaxis={"categoryorder": "total descending"},
                  title="Average Trip Distance by Weather Metric",
                  plot_bgcolor="rgba(0,0,0,0)",
                  width=800,
                  height=600)

fig.show()

### Timeseries Line Chart of How Rideshare Frequency Changes with Weather

Returning to our third question of "Do changes in weather affect ride frequency?", let's create a line chart of ride frequency and average temperature.

In [147]:
dfplot3 = dfmerge.copy()
dfplot3['week'] = dfplot3['DATE'].dt.isocalendar().week
dfplot3['year'] = dfplot3['DATE'].dt.year
dfplot3['week_year'] = dfplot3['year'].astype(str) + 'CW' + dfplot3['week'].astype(str)
for week in dfplot3['week_year'].unique():
  if len(week)==7:
    dfplot3.replace('{}CW{}'.format(week[0:4],week[-1]),'{}CW0{}'.format(week[0:4],week[-1]),inplace=True)

df_q3_time = dfplot3.groupby('week_year').mean()
df_q3_time.head()
subfig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig1 = px.line(
    df_q3_time,
    x=df_q3_time.index,
    y="ride_count",
)
fig1.for_each_trace(lambda t: t.update(name = 'Rideshare Frequency'))
fig1.update_traces(showlegend = True)
fig2 = px.line(
    df_q3_time,
    x=df_q3_time.index,
    y="TAVG"
)
fig2.for_each_trace(lambda t: t.update(name = 'Average Temperature'))
fig2.update_traces(showlegend = True)

# Combine Plots
fig2.update_traces(yaxis="y2")
subfig.add_traces(fig1.data + fig2.data)

#Add shaded annotations
subfig.add_vrect(x0='2020CW23',
                 x1='2020CW44',
                 fillcolor="Violet",
                 line_width=0,
                 opacity=0.1)
subfig.add_annotation(x='2020CW34',y=170000,text='Summer Heat',showarrow=False)

subfig.add_vrect(x0='2021CW50',
                 x1='2022CW08',
                 fillcolor="LightSeaGreen",
                 line_width=0,
                 opacity=0.15)
subfig.add_annotation(x='2022CW03',y=240000,text='Winter<br>Freeze',showarrow=False)

subfig.add_vrect(x0='2020CW12',
                 x1='2020CW12',
                 fillcolor="black",
                 line_dash='dash',
                 annotation_text='COVID-19 Pandemic Starts')

# Clean up Chart Layout
subfig.update_layout(title="Rideshare Frequency by Average Temperature (Weekly)",
                     xaxis={"title": "Time"},
                     yaxis={"title": "Ride Frequency"},
                     yaxis2={"title": "Average Temperature (deg F)"},
                     plot_bgcolor="rgba(0,0,0,0)")

subfig.for_each_trace(lambda t: t.update(line=dict(color=t.marker.color)))
subfig.update_layout(legend=dict(
    yanchor="top",
    y=1.2,
    xanchor="right",
    x=1
))
subfig.update_layout(xaxis_tick0 = df_q3_time.index[0], xaxis_dtick=12)
subfig.show()


In [149]:
dfplot3 = dfmerge.copy()
dfplot3['week'] = dfplot3['DATE'].dt.isocalendar().week
dfplot3['year'] = dfplot3['DATE'].dt.year
dfplot3['week_year'] = dfplot3['year'].astype(str) + 'CW' + dfplot3['week'].astype(str)
for week in dfplot3['week_year'].unique():
  if len(week)==7:
    dfplot3.replace('{}CW{}'.format(week[0:4],week[-1]),'{}CW0{}'.format(week[0:4],week[-1]),inplace=True)

df_q3_time = dfplot3.groupby('week_year').mean()
df_q3_time.head()
subfig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig1 = px.line(
    df_q3_time,
    x=df_q3_time.index,
    y="ride_count",
)
fig1.for_each_trace(lambda t: t.update(name = 'Rideshare Frequency'))
fig1.update_traces(showlegend = True)
fig2 = px.line(
    df_q3_time,
    x=df_q3_time.index,
    y="SNOW"
)
fig2.for_each_trace(lambda t: t.update(name = 'Snowfall'))
fig2.update_traces(showlegend = True)

# Combine Plots
fig2.update_traces(yaxis="y2")
subfig.add_traces(fig1.data + fig2.data)

#Add shaded annotations
subfig.add_vrect(x0='2020CW12',
                 x1='2020CW12',
                 fillcolor="black",
                 line_dash='dash',
                 annotation_text='COVID-19 Pandemic Starts')

subfig.add_vrect(x0='2021CW50',
                 x1='2022CW11',
                 fillcolor="LightSeaGreen",
                 line_width=0,
                 opacity=0.15)
subfig.add_annotation(x='2021CW40',y=350000,text='Winter<br>Snowstorms',showarrow=False)

subfig.add_vrect(x0='2020CW52',
                 x1='2021CW08',
                 fillcolor="LightSeaGreen",
                 line_width=0,
                 opacity=0.15)
subfig.add_annotation(x='2020CW43',y=350000,text='Winter<br>Snowstorms',showarrow=False)


# Clean up Chart Layout
subfig.update_layout(title="Rideshare Frequency by Snowfall (Weekly)",
                     xaxis={"title": "Time"},
                     yaxis={"title": "Ride Frequency"},
                     yaxis2={"title": "Snowfall (in.)"},
                     plot_bgcolor="rgba(0,0,0,0)")

subfig.for_each_trace(lambda t: t.update(line=dict(color=t.marker.color)))
subfig.update_layout(legend=dict(
    yanchor="top",
    y=1.2,
    xanchor="right",
    x=1
))
subfig.update_layout(xaxis_tick0 = df_q3_time.index[0], xaxis_dtick=12)
subfig.show()

## Statement of Work

Page 11 comprises a statement that describes the contribution that each team member made to the project. You should explain how you collaborated or cooperated with each other. Page 11 can also be used for the inclusion of endnotes

## TO BE DELETED BEFORE SUBMISSION

In [ ]:
# We have a big dataframe where each row is a date
# Question: How do changes in weather affect price -- "fare_mean"
# "Changes in weather" -- what does this mean?

# SPLOM?

# groups the data by quarter... not really sure how useful this is for our question.
# dfmergeQuarters = dfmerge.groupby(dfmerge['DATE'].dt.to_period('Q')).mean()

fig = px.scatter_matrix(dfmerge,
                        dimensions=[
                            "TAVG", "TMIN", "TMAX",
                            "PRCP", "SNOW",
                            "fare_mean", "ride_count"
                        ])
fig.show()

< team number >-< uniqname >-< uniqname >[-< uniqname >]_[YYYY][fall | winter].zip

Ex: 00-arwhyte-cteplovs-2023winter.zip

*Remember, the project report must be no greater than 11 pages in length. Keep this in mind if you are planning to generate the PDF from a Jupyter notebook. We **strongly recommend** that you use Google Docs or Slides, Microsoft Word or Powerpoint, or some other word processing package to generate your final PDF.*

“src” folder contains jupyter notebooks/python scripts
“src/data” folder contains the data you used

If the data is greater than 10 MB - make a sample file containing the first 100 records!
